## Case Ton | Análise de dados

#### Realizado por: Brena Arcênio Moreira

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.express as px


### Importação e tratamento das bases de dados antes das análises

In [ ]:
df1= pd.read_csv('cases.csv')
df2= pd.read_csv('creds.csv')

In [ ]:
#Com o resultado dessa linha podemos identificar que a principal fonte de dados de identificação do cliente possui 
#diversos valores vazios o que é bastante prejudicial em análises de dados pois são informações que não conseguimos
#mapear e por isso iremos limpar esses dados

df1.isnull().sum()
df2.isnull().sum()

df1= df1.dropna(subset=['Id'])
df2= df2.dropna(subset=['accountid'])

In [ ]:
# Analisando a tabela de chamados identifiquei que temos motivos macros dentro do assunto da ligação
# esses motivos macros irão auxiliar para fazermos analises futuramente.

new = df1["assunto"].str.split(":", n = 1, expand = True)

# Criando a Nova Coluna "First Name" com o new[0]
df1["Motivo_macro"]= new[0] 

# Criando a Nova Coluna "Last Name" com o new[1]
df1["Motivo_micro"]= new[1] 


In [ ]:
df1.drop(columns =["Unnamed: 0"], inplace = True) 
df2.drop(columns =["Unnamed: 0"], inplace = True) 

In [ ]:
# Aqui criamos o mes referencia para analisarmos em espaço de datas maiores 
df1['date_ref'] = pd.to_datetime(df1['date_ref'],dayfirst = True )
df1['contact_cohort']=df1['date_ref'].dt.strftime('%m/%Y')

df2['cred_date'] = pd.to_datetime(df2['cred_date'],dayfirst = True )
df2['account_cohort']=df2['cred_date'].dt.strftime('%m/%Y')

In [ ]:
df1['contact_cohort']=pd.to_datetime(df1['contact_cohort'],dayfirst = True )

#### vamos às questões:
1.	Foi implementada no dia 1 de agosto uma funcionalidade nova no site do Ton que acredita que irá diminuir a quantidade de chamados recebidos. Avalie se a implementação teve um impacto estatisticamente relevante!


In [ ]:
#primeiro vamos olhar a distribuição de ligações por mês:
ligacao_por_mes=df1.groupby('contact_cohort',as_index= False).count()
ligacao_por_mes=ligacao_por_mes[['contact_cohort','Id']]
display(ligacao_por_mes)

In [ ]:
fig=px.bar(ligacao_por_mes,x='contact_cohort',y='Id')
fig.show()

In [ ]:
#abrindo para verificarmos por dia: 
#primeiro vamos olhar a distribuição de ligações por mês:
ligacao_por_dia=df1.groupby('date_ref',as_index= False).count()
ligacao_por_dia=ligacao_por_dia[['date_ref','Id']]
fig=px.bar(ligacao_por_dia,x='date_ref',y='Id')
fig.show()

# Podemos verificar que há uma diminuição, mas nada que seja visualmente fora da curva

In [ ]:
# Vamos utilizar um cálculo de retorno para verificar se a diferença para o mês anterior foi significativa estatisticamente
aux1=df1.groupby('contact_cohort').count()
aux1=aux1['Id']


var_mom=((aux1-aux1.shift(1))/ aux1.shift(1))

var_mom

In [ ]:
# Não temos uma diminuição signficativa no mês de agosto, tendo o número de ligações aumentado logo após em setembro. 
#Em outubro sim temos uma queda um pouco mais significativa 

2.Foi notado pelo nosso time que clientes recem credenciados possuem um comportamento específico em relação à quantidade de chamados realizados logo após seu credenciamento. Identifique e prepare alguma forma de visualizar essa diferença notada pelos agentes.

In [ ]:
#Juntando as duas bases
aux2= df1.merge(df2,how='left',on='accountid')


In [ ]:
aux2['quantidade_dias']=((aux2['date_ref'] - aux2['cred_date'])/np.timedelta64(1 , 'D')).astype(int)

In [ ]:
aux3=aux2[['quantidade_dias','Motivo_macro','Id']].groupby(['quantidade_dias','Motivo_macro'],as_index = False).count()
aux3=aux3.sort_values(by=['quantidade_dias','Id'],ascending=True )
fig=px.bar(aux3,x='quantidade_dias',y='Id',color='Motivo_macro')
fig.show()

In [ ]:
## De fato há um maior incidente de ligações principalmente nos 15 primeiros dias após credenciamento.

3.	Para conseguir melhorar a qualidade do nosso atendimento ao cliente, foi proposto que a alguns chamados tenham prioridade de atendimento. Levando em consideração o motivo dos chamados, como você estabeleceria essa prioridade da fila de espera? Sugestão: Proponha uma clusterização da base de clientes que julgue atender da melhor forma a necessidade apresentada.

In [ ]:
# como ja proposto aqui separamos os clientes por motivo macro sendo ele utilizado para nossa clusterização. 
#Vamos ver quais motivos tem mais chamados
ligacao_por_motivo=aux2.groupby('Motivo_macro',as_index= False).count()
ligacao_por_motivo=ligacao_por_motivo[['Motivo_macro','Id']]
ligacao_por_motivo=ligacao_por_motivo.sort_values(by=['Id'],ascending=False )
fig=px.bar(ligacao_por_motivo,x='Motivo_macro',y='Id')
fig.show()

In [ ]:
#Abrindo em mais detalhes essas ligações
ligacao_por_motivo=aux2[['Motivo_macro','Motivo_micro','Id']].groupby(['Motivo_macro','Motivo_micro'],as_index = False).count()
ligacao_por_motivo=ligacao_por_motivo.sort_values(by=['Id'],ascending=False )
fig=px.bar(ligacao_por_motivo,x='Motivo_macro',y='Id',color='Motivo_micro')
fig.show()